In [4]:
import pandas as pd
import numpy as np
import matplotlib
import glob
import os
import re
from datetime import datetime, timedelta

### Load data

In [5]:
def load_and_concat_csv(folder_path, chunksize=None):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    df_list = []

    for filename in all_files:
        try:
            # Read the CSV file, potentially in chunks
            df_chunks = pd.read_csv(filename, chunksize=chunksize, 
                                    low_memory=False, encoding='utf-8')
            
            if chunksize:
                # If reading in chunks, concatenate the chunks
                df = pd.concat(df_chunks, ignore_index=True)
            else:
                df = next(df_chunks)
            
            df['source_file'] = os.path.basename(filename)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading file {filename}: {str(e)}")

    combined_df = pd.concat(df_list, ignore_index=True, sort=False)
    
    # Handle missing values
    #combined_df = combined_df.fillna('Unknown')  # or use another appropriate method
    
    return combined_df

In [3]:
folder_path = "../output/"
result_df = load_and_concat_csv(folder_path, chunksize=100000)

print(result_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8009673 entries, 0 to 8009672
Data columns (total 23 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Rank                               object 
 1   Performance                        object 
 2   Surname, first name                object 
 3   Club                               object 
 4   Nat.                               object 
 5   YOB                                object 
 6   M/F                                object 
 7   Rank M/F                           float64
 8   Cat                                object 
 9   Cat. Rank                          float64
 10  Avg.Speed km/h                     object 
 11  Age graded performance             object 
 12  Runner ID                          object 
 13  Event                              object 
 14  Date                               object 
 15  Distance                           object 
 16  Finishers         

In [21]:
def sample_csv(input_file, output_file, n, random_state=None):
    """
    Sample n rows from a CSV file and save to a new CSV file.
    
    :param input_file: Path to the input CSV file
    :param output_file: Path to save the output CSV file
    :param n: Number of rows to sample
    :param random_state: Seed for random number generator (optional)
    """
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Sample n rows
    sampled_df = df.sample(n=n, random_state=random_state)
    
    # Save the sampled DataFrame to a new CSV file
    sampled_df.to_csv(output_file, index=False)
    
    print(f"Sampled {n} rows from {input_file} and saved to {output_file}")

# Usage
input_file = '../output/all_events_data_2024.csv'
output_file = '2024_sample.csv'
sample_size = 500 

sample_csv(input_file, output_file, sample_size)

Sampled 500 rows from /Users/riz/Projects/duvdata/output/all_events_data_2024.csv and saved to 2024_sample.csv


### Clean data/Feature eng

In [63]:
df = pd.read_csv("../tmp.csv")
#df[df['Event ID']==105226]

In [64]:
def split_runner_name(data):
    data[['Surname', 'First Name']] = data['Surname, first name'].str.split(',', expand=True)
    original_name_column = 'Original name\nSurname, first name'
    if original_name_column in data.columns:
        data['First Name'] = data['First Name'].fillna(data[original_name_column])
        data['Surname'] = data['Surname'].fillna(data[original_name_column])
    return data

def parse_performance(data, column_name, time_column, distance_column):
    time_regex = re.compile(r'(?:(\d+)d )?(\d{1,2}):(\d{2}):(\d{2}) h')
    distance_regex = re.compile(r'(\d+\.?\d*) km')
    
    def parse_entry(entry):
        if 'km' in entry:
            match = distance_regex.search(entry)
            return (None, float(match.group(1)) if match else None)
        else:
            match = time_regex.search(entry)
            if match:
                days, hours, minutes, seconds = match.groups(default='0')
                total_seconds = timedelta(days=int(days), hours=int(hours), minutes=int(minutes), seconds=int(seconds)).total_seconds()
                return (total_seconds, None)
        return (None, None)
    
    data[time_column], data[distance_column] = zip(*data[column_name].apply(parse_entry))
    return data

def split_distance_column(df):
    distance_pattern = re.compile(r'(\d+\.?\d*)\s*(km|mi|h)')
    race_type_pattern = re.compile(r'(\d+\.?\d*\s*(km|mi|h))\s*(.*)')
    
    def split_distance(entry):
        distance_match = distance_pattern.search(entry)
        race_type_match = race_type_pattern.search(entry)
        
        if distance_match:
            distance = distance_match.group(0)
            unit = distance_match.group(2)
            race_type = 'Time' if unit == 'h' else 'Distance'
        else:
            distance = None
            race_type = None
        
        terrain = race_type_match.group(3).strip() if race_type_match and len(race_type_match.groups()) > 2 else None
        
        return distance, terrain, race_type
    
    df['Distance/Time'], df['Terrain'], df['Event Type'] = zip(*df['Distance'].apply(split_distance))
    return df

def convert_miles_to_km(entry):
    if pd.isna(entry):
        return None
    match = re.match(r'(\d+\.?\d*)(mi|km)', str(entry))
    if match:
        distance, unit = float(match.group(1)), match.group(2)
        return distance * 1.6 if unit == 'mi' else distance
    return None

def standardize_terrain(terrain):
    terrain = str(terrain).lower()
    if 'trail' in terrain:
        return 'trail'
    elif 'road' in terrain:
        return 'road'
    elif 'track' in terrain:
        return 'track'
    else:
        return 'other'


def parse_date_range(date_str):
    if pd.isna(date_str):
        return None
    
    date_str = str(date_str).strip()
    
    # If it's already in the correct format, return as is
    if re.match(r'\d{2}\.\d{2}\.\d{4}', date_str):
        return date_str
    
    # If it's a single date without year, add the current year
    if re.match(r'\d{1,2}\.\d{1,2}\.?$', date_str):
        return f"{date_str.rstrip('.')}1970"  # Using 1970 as a placeholder year
    
    # If it's a date range
    if '-' in date_str:
        start, end = date_str.split('-')
        start = start.strip()
        end = end.strip()
        
        # Extract day, month, and year components
        start_parts = re.findall(r'\d+', start)
        end_parts = re.findall(r'\d+', end)
        
        # Ensure we have at least day and month for start date
        if len(start_parts) < 2:
            return None  # Return None if format is unexpected
        
        # Get year from end date if available, otherwise use current year
        year = end_parts[-1] if len(end_parts) == 3 else str(datetime.now().year)
        
        # Construct the full start date
        return f"{start_parts[0].zfill(2)}.{start_parts[1].zfill(2)}.{year}"
    
    # If it's a single date with year
    if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_str):
        parts = date_str.split('.')
        return f"{parts[0].zfill(2)}.{parts[1].zfill(2)}.{parts[2]}"
    
    # If we can't parse the date, return None
    return None


def extract_location(event):
    match = re.search(r'\((\w+)\)$', event)
    return match.group(1) if match else 'Unknown'

def categorize_age_group(age):
    if pd.isna(age):
        return 'Unknown'
    elif age < 20:
        return 'Under 20'
    elif 20 <= age < 30:
        return '20-29'
    elif 30 <= age < 40:
        return '30-39'
    elif 40 <= age < 50:
        return '40-49'
    elif 50 <= age < 60:
        return '50-59'
    elif 60 <= age < 70:
        return '60-69'
    else:
        return '70+'

def add_age_group(df):
    df['Age Group'] = df['Age'].apply(categorize_age_group)
    return df

def add_runner_statistics(df):
    # Sort the dataframe by Runner ID and Date
    df = df.sort_values(['Runner ID', 'Date'])
    
    # Group by Runner ID and calculate statistics
    grouped = df.groupby('Runner ID')
    
    # Number of races (Experience Level)
    df['Race Count'] = grouped.cumcount() + 1
    
    # Determine the distance to use for cumulative calculation
    df['Distance For Cumulative'] = df.apply(
        lambda row: row['Distance Finish'] if row['Event Type'] == 'Time' else row['Distance KM'],
        axis=1
    )
    
    # Cumulative sum of Distance
    df['Cumulative Distance KM'] = grouped['Distance For Cumulative'].cumsum()
    
    # Rolling average of Winner Percentage
    df['Avg Winner Percentage'] = grouped['Winner Percentage'].transform(
        lambda x: x.expanding().mean()
    )
    
    # Remove the temporary column
    df = df.drop('Distance For Cumulative', axis=1)
    
    return df

def extract_finishers(df):
    def parse_finishers(finishers_str):
        match = re.match(r'(\d+)\s*\((\d+)\s*M,\s*(\d+)\s*F\)', finishers_str)
        if match:
            total = int(match.group(1))
            male = int(match.group(2))
            female = int(match.group(3))
            return total, male, female
        else:
            return None, None, None

    df[['Total Finishers', 'Male Finishers', 'Female Finishers']] = df['Finishers'].apply(parse_finishers).apply(pd.Series)
    return df


def add_elevation_gain_per_km(df):
    # Calculate elevation gain per km
    df['Elevation Gain per KM'] = df.apply(
        lambda row: row['Elevation Gain'] / row['Distance KM'] 
        if pd.notnull(row['Elevation Gain']) and pd.notnull(row['Distance KM']) and row['Distance KM'] != 0 
        else np.nan, 
        axis=1
    )
    
    # Calculate the median elevation gain per km for each terrain type
    median_elevation_by_terrain = df.groupby('Terrain')['Elevation Gain per KM'].median()
    
    # Fill NaN values with the median for the corresponding terrain type
    df['Elevation Gain per KM'] = df.apply(
        lambda row: median_elevation_by_terrain[row['Terrain']] 
        if pd.isnull(row['Elevation Gain per KM']) and row['Terrain'] in median_elevation_by_terrain
        else row['Elevation Gain per KM'],
        axis=1
    )
    
    return df

def calculate_winner_percentage(df):
    # For distance races (where Event Type is 'Distance')
    distance_mask = df['Event Type'] == 'Distance'
    df.loc[distance_mask, 'Winner Percentage'] = (1 - df.loc[distance_mask, 'Time Seconds Winner'] / df.loc[distance_mask, 'Time Seconds Finish']).round(2)

    # For time races (where Event Type is 'Time')
    time_mask = df['Event Type'] == 'Time'
    df.loc[time_mask, 'Winner Percentage'] = (1 - df.loc[time_mask, 'Distance Finish'] / df.loc[time_mask, 'Distance Winner']).round(2)

    return df


In [65]:
def clean_data(df):
    df = split_runner_name(df)
    df = split_distance_column(df)
    df = parse_performance(df, 'Performance', 'Time Seconds Finish', 'Distance Finish')
    df = parse_performance(df, 'Winner Time', 'Time Seconds Winner', 'Distance Winner')
    df = calculate_winner_percentage(df)
    df = extract_finishers(df)
    
    df['Terrain'] = df['Terrain'].apply(standardize_terrain)
    df['Distance KM'] = df['Distance/Time'].apply(convert_miles_to_km).round(0)
    df['Finish Percentage'] = (df['Rank'] / df['Total Finishers']).round(2)
    df['Distance KM'] = df['Distance KM'].replace(0, pd.NA)
    df['Average Speed'] = df['Time Seconds Finish'] / df['Distance KM']
    df['Race Location'] = df['Event'].apply(extract_location)

    df['Date'] = pd.to_datetime(df['Date'].apply(parse_date_range), format='%d.%m.%Y')
    df['YOB'] = pd.to_numeric(df['YOB'], errors='coerce').astype('Int64')
    df['Avg.Speed km/h'] = df['Avg.Speed km/h'].astype(float)
    df['Elevation Gain'] = pd.to_numeric(df['Elevation Gain'].replace({'Hm': '', 'm': ''}, regex=True), errors='coerce')

    df = df.fillna({'M/F': 'Unknown', 'Cat': 'Unknown'})
    df['Club'] = df['Club'].str.strip().str.replace(r'[^\w\s]', '', regex=True)
    df['Nat.'] = df['Nat.'].str.strip().str.upper()
    df['Age'] = df['Date'].dt.year - df['YOB']
    df = add_age_group(df)
    
    df = add_runner_statistics(df)
    df = add_elevation_gain_per_km(df)
    
    return df

In [66]:
# Apply the cleaning function to your DataFrame
df_clean = clean_data(df)
df_clean.head()

,Rank,Performance,"Surname, first name",Club,Nat.,YOB,M/F,Rank M/F,Cat,Cat. Rank,...,Distance KM,Finish Percentage,Average Speed,Race Location,Age,Age Group,Race Count,Cumulative Distance KM,Avg Winner Percentage,Elevation Gain per KM
624,26,7:12:47 h,"Irrgang, Michael",LG Ultralauf,GER,1964,M,22,M60,3,...,54.0,0.46,480.87037,GER,60,60-69,1,54.000,0.38,32.407407
566,21,131.088 km,NaN,NaN,FRA,1959,M,17,M65,1,...,NaN,0.40,NaN,FRA,65,60-69,1,131.088,0.36,24.500000
647,49,8:36:09 h,"Risch, Joachim",100 Marathon Club,GER,1955,M,37,M65,1,...,54.0,0.86,573.50000,GER,69,60-69,1,54.000,0.48,32.407407
303,21,13:45:03 h,NaN,NaN,ITA,1962,M,17,M60,2,...,100.0,0.34,495.03000,ITA,62,60-69,1,100.000,0.31,24.500000
568,23,127.985 km,NaN,Perigueux Running,FRA,1953,F,5,W70,1,...,NaN,0.43,NaN,FRA,71,70+,1,127.985,0.38,24.500000


In [67]:
columns_to_keep = ['Runner ID','First Name','Surname','Nat.','M/F','Age','Age Group','Cat','YOB','Race Count','Cumulative Distance KM','Avg Winner Percentage','Event ID','Event','Event Type','Date','Race Location','Elevation Gain','Elevation Gain per KM','Finishers','Total Finishers','Male Finishers','Female Finishers','Rank','Rank M/F','Cat. Rank','Finish Percentage','Winner Percentage','Distance/Time','Distance KM','Terrain','Time Seconds Finish','Distance Finish','Average Speed','Avg.Speed km/h']
df_clean = df_clean[columns_to_keep]

In [68]:
df_clean.to_csv('../tmp_clean.csv',index=False)